In [ ]:
import pandas as pd

In [ ]:
file = 'legacy_data/Verhandlungsschriften_neu.xlsx'
res_type_scheme, _ = SkosConceptScheme.objects.get_or_create(dc_title='res_type')
df = pd.read_excel(file, dtype={'Nummer': str}).fillna('False')
df['Teil von Bestand'] = df[['Archiv', 'Bestand']].apply(lambda x: '; '.join(x), axis=1)
df['abstract'] = df[['Anfrage', 'Antwort der Kommission']].apply(lambda x: ' ANTWORT DER KOMMISSION: '.join(x), axis=1)

In [ ]:
archiv, _ = Institution.objects.get_or_create(
    written_name='Österreichisches Staatsarchiv / Archiv der Republik',
    abbreviation="ÖStA/AdR",
    institution_type="Archiv"
)
res_type_scheme, _ = SkosConceptScheme.objects.get_or_create(dc_title='res_type')
res_type, _ = SkosConcept.objects.get_or_create(pref_label='Verhandlungsschrift')
res_type.scheme.add(res_type_scheme)

In [ ]:
df

In [ ]:
for i, row in df.iterrows():
    item = None
    signatur = ",".join(
            [
                "Bestand: {}".format(str(row.get('Bestand', default=""))),
                "Nr: {}".format(str(row.get('Nummer', default="-"))), 
                "Unterpunkt: {}".format(str(row.get('Unterpunkt', default=""))),
                "Seite: {}".format(str(row.get('Seite', default=""))),
                "[internal-id]: {}".format(i)
            ]
        )
    try:
        item, _ = ArchResource.objects.get_or_create(
            signature=signatur
        )
    except Exception as e:
        print(e)
    if item:
        if row['Anfrage'] != 'False':
            item.title = row['Anfrage'][:150]
        if row['abstract'] != 'False':
            item.abstract = row['abstract']
        if row['Start Date'] != 'False':
            try:
                item.not_before = pd.to_datetime(row['Start Date'])
            except Exception as e:
                print(e)
        if row['Ort'] != 'False':
            for x in row['Ort'].split(';'):
                try:
                    pl = Place.objects.get(name=x.strip())
                except Exception as e:
                    pl = None
                    try:
                        other_name = x.strip().split('(')[0]
                        pl = Place.objects.get(name=other_name)
                    except:
                        pl = None
                        print(e, x.strip())
                if pl:
                    item.mentioned_place.add(pl)
        if row['Institution'] != 'False':
            for x in row['Institution'].split(';'):
                try:
                    pl = Institution.objects.get(written_name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.mentioned_inst.add(pl)
        if row['Person'] != 'False':
            for x in row['Person'].split(';'):
                try:
                    pl = Person.objects.get(written_name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.mentioned_person.add(pl)
        item.archiv = archiv
        item.res_type = res_type
        item.save()
            
        

In [ ]:
ArchResource.objects.filter(signature__startswith='ÖStA/AdR|Staatskanzlei').delete()